In [705]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

# Retrieve COVID data from ECDC 

In [706]:
covid_url = "https://opendata.ecdc.europa.eu/covid19/casedistribution/json/"
import ssl
ssl._create_default_https_context = ssl._create_unverified_context
import json
import urllib

In [707]:
covid_json_unformated = urllib.request.urlopen(covid_url).read().decode("utf-8")
covid_json = json.loads(covid_json_unformated)
cdf = pd.DataFrame(covid_json['records'])

In [708]:
cdf.sample(10)

,dateRep,year_week,cases_weekly,deaths_weekly,countriesAndTerritories,geoId,countryterritoryCode,popData2019,continentExp,notification_rate_per_100000_population_14-days
2917,08/06/2020,2020-23,9094,278,Egypt,EG,EGY,100388076.0,Africa,16.75
10130,28/12/2020,2020-52,0,0,Vanuatu,VU,VUT,299882.0,Oceania,0
6636,14/09/2020,2020-37,1551,16,Myanmar,MM,MMR,54045422.0,Asia,4.12
2318,30/03/2020,2020-13,140,1,Cote_dIvoire,CI,CIV,25716554.0,Africa,0.63
1161,23/03/2020,2020-12,2,0,Bhutan,BT,BTN,763094.0,Asia,
4154,11/05/2020,2020-19,560,4,Guinea,GN,GIN,12771246.0,Africa,8.24
581,20/07/2020,2020-29,724,2,Austria,AT,AUT,8858775.0,Europe,14.7
2324,11/01/2021,2021-01,7265,331,Croatia,HR,HRV,4076246.0,Europe,367.42
9296,27/01/2020,2020-04,5,0,Taiwan,TW,CNG1925,23773881.0,Asia,0.02
187,10/08/2020,2020-32,31,0,Andorra,AD,AND,76177.0,Europe,77.45


Rename columns to something more Pythonian. If you think they look already great, then at least rename `notification_rate_per_100000_population_14-days` to `14d-incidence`

In [709]:
cdf = cdf.rename(columns={
    'dateRep': 'date_reported',
    'year_week': 'year_and_week',
    'cases_weekly': 'cases_per_week',
    'deaths_weekly': 'deaths_per_week',
    'countriesAndTerritories': 'country',
    'geoId': 'iso_3166_1_alpha2',
    'countryterritoryCode': 'iso_3166_1_alpha3',
    'popData2019': 'population_2019',
    'continentExp': 'continent',
    'notification_rate_per_100000_population_14-days': 'incidence_14_days'
})

Identify which columns have not been casted to an appropriate type during loading!

In [540]:
cdf.dtypes

date_reported         object
year_and_week         object
cases_per_week         int64
deaths_per_week        int64
country               object
iso_3166_1_alpha2     object
iso_3166_1_alpha3     object
population_2019      float64
continent             object
incidence_14_days     object
dtype: object

In [541]:
cdf.country = cdf.country.astype('category')
cdf.iso_3166_1_alpha2 = cdf.iso_3166_1_alpha2.astype('category')
cdf.iso_3166_1_alpha3 = cdf.iso_3166_1_alpha3.astype('category')
cdf.continent = cdf.continent.astype('category')

In [542]:
cdf.cases_per_week = cdf.cases_per_week.astype('Int64')
cdf.deaths_per_week = cdf.deaths_per_week.astype('Int64')

In [543]:
any((cdf.population_2019 - cdf.population_2019.round()).abs() > 1e-10)

False

In [544]:
cdf.population_2019 = cdf.population_2019.astype('Int64')

In [712]:
cdf.incidence_14_days = pd.to_numeric(cdf.incidence_14_days.replace('^\s*$', pd.NA, regex=True))

We did not cover datetime objects in pandas, however they are quite powerful!

Try:

In [546]:
cdf.date_reported = pd.to_datetime(cdf.date_reported)

Now you can treat the column as a datetime objects using `df[col].dt` , e.g. https://docs.python.org/3/library/datetime.html#datetime.date.year

In [547]:
cdf.dtypes

date_reported        datetime64[ns]
year_and_week                object
cases_per_week                Int64
deaths_per_week               Int64
country                    category
iso_3166_1_alpha2          category
iso_3166_1_alpha3          category
population_2019               Int64
continent                  category
incidence_14_days           float64
dtype: object

In [548]:
cdf['date_reported'].dt.day.head()

0     2
1    25
2    18
3     1
4     1
Name: date_reported, dtype: int64

Create a new column `deltaTime_since_start_of_recording`

In [549]:
cdf[['deltaTime_since_start_of_recording']] = cdf.date_reported - cdf.date_reported.min()

Create histograms for different columns or describe the df. Can you spot the inconsistency in the data? Fix it! :)

In [550]:
cdf.describe()

,cases_per_week,deaths_per_week,population_2019,incidence_14_days,deltaTime_since_start_of_recording
count,1.043300e+04,10433.000000,1.040800e+04,10195.000000,10433
mean,9.915481e+03,214.363366,4.015934e+07,85.263560,219 days 03:27:51.810600976
std,5.918195e+04,1017.496922,1.508375e+08,201.992927,126 days 06:36:24.926596194
min,-3.864000e+03,-875.000000,8.150000e+02,-132.600000,0 days 00:00:00
25%,9.000000e+00,0.000000,1.269670e+06,0.920000,125 days 00:00:00
50%,1.910000e+02,2.000000,7.169456e+06,8.470000,213 days 00:00:00
75%,2.615000e+03,42.000000,2.851583e+07,71.225000,304 days 00:00:00
max,1.782792e+06,23518.000000,1.433784e+09,4343.440000,665 days 00:00:00


In [551]:
cdf.cases_per_week < 0

0        False
1        False
2        False
3        False
4        False
         ...  
10428    False
10429    False
10430    False
10431    False
10432    False
Name: cases_per_week, Length: 10433, dtype: boolean

In [552]:
cdf.cases_per_week.mask(cdf.cases_per_week < 0, pd.NA, inplace=True)
cdf.deaths_per_week.mask(cdf.deaths_per_week < 0, pd.NA, inplace=True)
cdf.incidence_14_days.mask(cdf.incidence_14_days < 0, pd.NA, inplace=True)

Identify those countries (grouped by continent) which showed the most drastic increase most drastic and decrease of the `14d-incidence` within the different years since recording. Visualize intuitively!

In [713]:
weekly_reports = cdf[['year_and_week', 'country', 'date_reported', 'cases_per_week', 'deaths_per_week', 'incidence_14_days']] \
    .set_index(['year_and_week', 'country']) \
    .groupby(level=[0, 1]) \
    .first() \
    .reset_index()

weekly_reports[['year', 'week_of_year']] = weekly_reports.year_and_week.str.split('-', expand=True).astype('int')

weekly_reports.drop(columns='year_and_week', inplace=True)
weekly_reports.set_index(['year', 'week_of_year'], inplace=True)

incidence_difference = weekly_reports.groupby('country') \
    .incidence_14_days \
    .rolling(2) \
    .apply(lambda series: series.iloc[1] - series.iloc[0]) \
    .reorder_levels([1, 2, 0])

weekly_reports.reset_index(inplace=True)
weekly_reports.set_index(['year', 'week_of_year', 'country'], inplace=True)

weekly_reports[['incidence_diff']] = incidence_difference

country_info = cdf[['country', 'iso_3166_1_alpha2', 'iso_3166_1_alpha3', 'population_2019', 'continent']] \
    .set_index('country') \
    .groupby(level=0) \
    .first()

Which country showed the highest/lowest fluctuation in `14d-incidence` within a year?

Create a line plot showing the `14-incidence` for all European countries. Use `groupby` operation to generate the data list for the plotly plot. 

Create a smoothed version of the `14d-incidence` by averaging 3 months.

Create a radial plot of death rate / 100000 people (see popData2019), where one year completes a circle, i.e. 360˚. Visualize the recored years for Italy, Germany, Sweden and Greece. Hint you might need to turn the dateTime into `day within the year` (%j) and adjust 365 to 360 degrees. 

Optional: Find "regular" mortality rates for those countries and visualize it in the plot as well.